<a href="https://colab.research.google.com/github/JiazanShi/PUI-project/blob/main/Copy_of_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import os
import sys

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, Conv1D, MaxPooling1D, Embedding, Conv2D, MaxPool2D
from keras.models import Model
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim

import keras.regularizers

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:

def remove_punctuations(docs): #list of strings
    punctuations = '''!()[]{};:"\,<>'./?@#$%^&*_~-'''
    docs2=[];
    i=0;
    for sent in docs:
        s=""
        for x in sent:
            if(x not in punctuations):
                s=s+x;
        docs2.append(s);


    docs=docs2;
    docs2=[]
    return docs


def convert_lowercase(docs): #list of strings
    docs2=[]
    for sent in docs:
        sent=sent.lower()
        docs2.append(sent);

    docs=docs2;
    docs2=[]
    return docs




def tokenize_data(docs): #list of strings
    docs2=[]
    for sent in docs:
        sent=word_tokenize(sent)
        docs2.append(sent);

    docs=docs2;
    docs2=[]
    return docs

def remove_stopwords(docs): #list of list of words as param
    stop_words=set(stopwords.words("english"));

    docs2=[]
    for sent in docs:
        s=[];
        for word in sent:
            if(word not in stop_words):
                s.append(word);
        docs2.append(s);

    docs=docs2;
    docs2=[]
    return docs


def make_string(docs): #list of list of words as param
    docs2=[]
    for sent in docs:
        s=""
        s=' '.join(sent)
        docs2.append(s);

    docs=docs2;
    docs2=[]
    return docs


def lemmatize(docs): #list of list of words as param
    lemmatizer=WordNetLemmatizer();
    docs2=[]
    for sent in docs:
        s=[];
        for word in sent:
            s.append(lemmatizer.lemmatize(word));
        docs2.append(s);

    docs=docs2;
    docs2=[]
    return docs


def remove_digits(docs): #list of strings
    docs2=[]
    for sent in docs:
        sent2=sent.split(' ');
        words=[]
        for x in sent2:
            flag=0;
            for c in x:
                if(c>='0' and c<='9'):
                    flag=1;
                    break;
                else:
                    pass;
            if(flag==0):
                words.append(x);
        words=' '.join(words);
        docs2.append(words)
    docs=docs2;
    docs2=[]
    return docs

In [ ]:
data=pd.read_csv('guns.csv')
data=data.drop(columns=['Unnamed: 0','text'])
data.head()

,stance,original_text,topic,quote
0,stance1,for,People_Should_Have_The_Right_To_Own_Guns,Yes! It's part of our constitutional right. W...
1,stance1,Yes,what-do-guns-mean-to-americans,"While I agree with your basic point, (I think)..."
2,stance1,For,Gun_Rights,"People need the right of defense, safty, a gun..."
3,stance1,Yes,should-loaded-and-concealed-guns-be-allowed-in...,If you stay in the city and are uncomfortable ...
4,stance1,Yes,does-owning-a-gun-make-you-safer,You said it all....


In [ ]:
data['text']=convert_lowercase(data['quote'])
data['text']=remove_digits(data['text'])
data['text']=remove_punctuations(data['text'])
data['text']=tokenize_data(data['text'])
data['text']=remove_stopwords(data['text'])
data['text']=lemmatize(data['text'])
data['text']=make_string(data['text'])
data.head()


,stance,original_text,topic,quote,text
0,stance1,for,People_Should_Have_The_Right_To_Own_Guns,Yes! It's part of our constitutional right. W...,yes part constitutional right well sane respon...
1,stance1,Yes,what-do-guns-mean-to-americans,"While I agree with your basic point, (I think)...",agree basic point think certainly come irratio...
2,stance1,For,Gun_Rights,"People need the right of defense, safty, a gun...",people need right defense safty gun people arg...
3,stance1,Yes,should-loaded-and-concealed-guns-be-allowed-in...,If you stay in the city and are uncomfortable ...,stay city uncomfortable firearm mean dont buy ...
4,stance1,Yes,does-owning-a-gun-make-you-safer,You said it all....,said


In [ ]:
X=data['text']
Y=data['stance']

In [ ]:
from transformers import BertTokenizer
print("Loading...")
tokenizer=BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=True)

Loading...


In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3)

In [ ]:
train_text=list(X_train)

import numpy as np
input_ids=[]
lengths=[]
for text in train_text:
  encoded_sent=tokenizer.encode(text,add_special_tokens=True)
  input_ids.append(encoded_sent)
  lengths.append(len(encoded_sent))

Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
y1=list(Y_train)
y_train=[]
for i in range(len(Y_train)):
  if(y1[i]=='stance1'):
    y_train.append(0)
  elif(y1[i]=='stance2'):
    y_train.append(1)

In [ ]:
labels=y_train

In [ ]:
print(max(lengths))
print(min(lengths))
print(np.median(lengths))

553
2
54.5


In [ ]:
MAX_LEN=512
input_ids=pad_sequences(input_ids,maxlen=MAX_LEN,dtype='long',value=0, truncating='post',padding='post')

In [ ]:
# create attention masks
#if token id is 0, then its padded, set the mask to 0
attention_masks=[]
for tweet in input_ids:
  att_mask=[int(token_id >0) for token_id in tweet]
  attention_masks.append(att_mask)

In [ ]:
print(attention_masks[0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print(len(input_ids))

748


In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,labels, random_state=42, test_size=0.1)
train_masks, validation_masks,_,_=train_test_split(attention_masks,labels, random_state=42, test_size=0.1)

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
import torch

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset , DataLoader, RandomSampler, SequentialSampler

batch_size=16

train_data=TensorDataset(train_inputs, train_masks, train_labels)
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data,sampler=train_sampler, batch_size=batch_size)

validation_data=TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler=SequentialSampler(validation_data)
validation_dataloader=DataLoader(validation_data,sampler=validation_sampler, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model=BertForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=3,
    output_attentions=False,
    output_hidden_states=False
)
     


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
#initial learning rate, adam_epsilon
optimizer=AdamW(model.parameters(), lr=0.0001, eps=1e-8)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs=4

total_steps=len(train_dataloader)*epochs
scheduler=get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    print(preds)
    print(labels)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

In [ ]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        #if step % 40 == 0 and not step == 0:
        # Calculate elapsed time in minutes.
        elapsed = format_time(time.time() - t0)
        
        # Report progress.
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch     0  of     43.    Elapsed: 0:00:00.
  Batch     1  of     43.    Elapsed: 0:00:02.
  Batch     2  of     43.    Elapsed: 0:00:04.
  Batch     3  of     43.    Elapsed: 0:00:05.
  Batch     4  of     43.    Elapsed: 0:00:06.
  Batch     5  of     43.    Elapsed: 0:00:08.
  Batch     6  of     43.    Elapsed: 0:00:09.
  Batch     7  of     43.    Elapsed: 0:00:11.
  Batch     8  of     43.    Elapsed: 0:00:12.
  Batch     9  of     43.    Elapsed: 0:00:14.
  Batch    10  of     43.    Elapsed: 0:00:15.
  Batch    11  of     43.    Elapsed: 0:00:17.
  Batch    12  of     43.    Elapsed: 0:00:18.
  Batch    13  of     43.    Elapsed: 0:00:20.
  Batch    14  of     43.    Elapsed: 0:00:21.
  Batch    15  of     43.    Elapsed: 0:00:23.
  Batch    16  of     43.    Elapsed: 0:00:24.
  Batch    17  of     43.    Elapsed: 0:00:26.
  Batch    18  of     43.    Elapsed: 0:00:27.
  Batch    19  of     43.    Elapsed: 0:00:29.
  Batch    20  of

In [ ]:
test_text=list(X_test)

input_ids=[]
lengths=[]
for t in test_text:
  encoded_sent=tokenizer.encode(t,add_special_tokens=True)
  input_ids.append(encoded_sent)
  lengths.append(len(encoded_sent))


y2=list(Y_test)
y_test=[]
for i in range(len(Y_test)):
  if(y2[i]=='stance1'):
    y_test.append(0)
  elif(y2[i]=='stance2'):
    y_test.append(1)

In [ ]:
MAX_LEN=512
input_ids=pad_sequences(input_ids,maxlen=MAX_LEN,dtype='long',value=0, truncating='post',padding='post')


# create attention masks
#if token id is 0, then its padded, set the mask to 0
attention_masks=[]
for tweet in input_ids:
  att_mask=[int(token_id >0) for token_id in tweet]
  attention_masks.append(att_mask)
     

In [ ]:
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(y_test)

In [ ]:
# Set the batch size.  
batch_size = 16

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 321 test sentences...
    DONE.


In [ ]:
y_pred=[]
for i in predictions:
  for k in i:
    y_pred.append(np.argmax(k))
     

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
print(f1_score(y_pred,y_test,average='macro'))
print(precision_score(y_pred,y_test,average='macro'))
print(recall_score(y_pred,y_test,average='macro'))
print(accuracy_score(y_pred,y_test))

0.5456194599051742
0.5456008583690988
0.5604864794928193
0.6791277258566978
